In [1]:
import re
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
feature_type = '2'

#1 -> just sentence
#2 -> all text

In [4]:
target_type = 1

#1 -> pos

In [5]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [6]:
train.head()

,text,sentence,snp,phenotype,ASSOCIATION,CONFIDENCE
0,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,nicotine metabolism,neutral,zero
1,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,fetal growth characteristics,positive,weak
2,OBJECTIVE: Maternal smoking during pregnancy i...,RESULTS: Among mothers who did not smoke durin...,rs1051730,fetal growth characteristic,negative,-
3,OBJECTIVE: Maternal smoking during pregnancy i...,Among mothers who continued smoking during pre...,rs1051730,head circumference,negative,-
4,OBJECTIVE: Maternal smoking during pregnancy i...,The T-allele of maternal rs1051730 was associa...,rs1051730,second and third trimester fetal femur length,positive,weak


In [7]:
train['label'] = train['CONFIDENCE']
test['label']  = test['CONFIDENCE']

In [8]:
train.dropna(subset=['sentence', 'text'], inplace=True)
test.dropna(subset=['sentence', 'text'], inplace=True)

In [9]:
train = train[train.ASSOCIATION == 'positive']
test  =  test[test.ASSOCIATION  == 'positive']

In [10]:
test['label'].value_counts()

weak        140
moderate     16
strong       14
Name: label, dtype: int64

In [11]:
train['label'].value_counts()

weak        375
strong      218
moderate    108
Name: label, dtype: int64

In [12]:
res = pd.read_excel('res.xlsx')

In [13]:
def vectorize_bow(df_X, cv):
    bow_X = cv.transform(df_X)
    bow_df = pd.DataFrame(data=bow_X.toarray(),
                          columns=[cv.get_feature_names_out()])
    return bow_df

def define_and_run_model(model, X_train, X_test, y_train):
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def show_result(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))
    print(accuracy_score(y_test, y_pred))
    
def save_res(model_name, feature_type, target_type, y_pred_sr, y_test_sr, idx):
    res_pre = precision_score(y_test_sr, y_pred_sr, average='weighted')
    res_rec = recall_score(y_test_sr, y_pred_sr, average='weighted')
    res_f1 = f1_score(y_test_sr, y_pred_sr, average='weighted')
    res.at[idx, 'model-name'] = model_name
    res.at[idx, 'feature-type'] = feature_type
    res.at[idx, 'target-type'] = target_type
    res.at[idx, 'accuracy'] = accuracy_score(y_test_sr, y_pred_sr)
    res.at[idx, 'precision'] = res_pre
    res.at[idx, 'recall'] = res_rec
    res.at[idx, 'f1-score'] = res_f1

In [14]:
possible_labels = train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

train['label'] = train.label.replace(label_dict)
test['label']  = test.label.replace(label_dict)

In [15]:
df = pd.concat([train, test])

In [16]:
bow_vectorizer = CountVectorizer(lowercase=False, ngram_range=(1, 1), max_df=.80, min_df=1)
bow_vectorizer.fit(df['text'])

CountVectorizer(lowercase=False, max_df=0.8)

In [17]:
train_bow_df  = vectorize_bow(list(train['text']), bow_vectorizer)
test_bow_df   = vectorize_bow(list(test['text']), bow_vectorizer)

In [18]:
X_train = train_bow_df
y_train = train['label']

X_test = test_bow_df
y_test = test['label']

In [19]:
X_train.shape

(701, 6622)

In [20]:
y_train.shape

(701,)

In [21]:
X_test.shape

(170, 6622)

In [22]:
y_test.shape

(170,)

In [23]:
train['ASSOCIATION'].value_counts()

positive    701
Name: ASSOCIATION, dtype: int64

In [24]:
test['ASSOCIATION'].value_counts()

positive    170
Name: ASSOCIATION, dtype: int64

In [25]:
%%time
smodel = SVC(kernel='rbf')
smodel.fit(X_train, y_train)
pred_values_svm = smodel.predict(X_test)
show_result(pred_values_svm, y_test)
save_res('svm', feature_type, target_type, pred_values_svm, y_test, len(res))

[[139   1   0]
 [ 11   3   0]
 [ 12   0   4]]
              precision    recall  f1-score   support

           0      0.858     0.993     0.921       140
           1      0.750     0.214     0.333        14
           2      1.000     0.250     0.400        16

    accuracy                          0.859       170
   macro avg      0.869     0.486     0.551       170
weighted avg      0.862     0.859     0.823       170

0.8588235294117647
Wall time: 2.71 s


In [26]:
%%time
lr = LogisticRegression(random_state=0,
                        penalty='l2',
                        solver='newton-cg',
                        C=1.2,
                        max_iter=20)
lr.fit(X_train, y_train)
pred_values_lr = lr.predict(X_test)
show_result(pred_values_lr, y_test)
save_res('LogisticRegression', feature_type, target_type, pred_values_lr, y_test, len(res))

[[81 57  2]
 [ 6  8  0]
 [ 9  3  4]]
              precision    recall  f1-score   support

           0      0.844     0.579     0.686       140
           1      0.118     0.571     0.195        14
           2      0.667     0.250     0.364        16

    accuracy                          0.547       170
   macro avg      0.543     0.467     0.415       170
weighted avg      0.767     0.547     0.616       170

0.5470588235294118
Wall time: 2.49 s


In [27]:
%%time
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pred_values_rf = model.predict(X_test)
show_result(pred_values_rf, y_test)
save_res('RandomForest', feature_type, target_type, pred_values_rf, y_test, len(res))

[[139   0   1]
 [ 12   2   0]
 [ 12   0   4]]
              precision    recall  f1-score   support

           0      0.853     0.993     0.917       140
           1      1.000     0.143     0.250        14
           2      0.800     0.250     0.381        16

    accuracy                          0.853       170
   macro avg      0.884     0.462     0.516       170
weighted avg      0.860     0.853     0.812       170

0.8529411764705882
Wall time: 3.26 s


In [28]:
%%time
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
pred_values_knn = model.predict(X_test)
show_result(pred_values_knn, y_test)
save_res('knn', feature_type, target_type, pred_values_knn, y_test, len(res))

[[116  24   0]
 [  8   6   0]
 [ 11   1   4]]
              precision    recall  f1-score   support

           0      0.859     0.829     0.844       140
           1      0.194     0.429     0.267        14
           2      1.000     0.250     0.400        16

    accuracy                          0.741       170
   macro avg      0.684     0.502     0.503       170
weighted avg      0.818     0.741     0.754       170

0.7411764705882353
Wall time: 186 ms


In [29]:
%%time
model = GaussianNB()
model.fit(X_train, y_train)
pred_values_nb = model.predict(X_test)
show_result(pred_values_nb, y_test)
save_res('GaussianNB', feature_type, target_type, pred_values_nb, y_test, len(res))

[[95 21 24]
 [ 7  6  1]
 [ 6  6  4]]
              precision    recall  f1-score   support

           0      0.880     0.679     0.766       140
           1      0.182     0.429     0.255        14
           2      0.138     0.250     0.178        16

    accuracy                          0.618       170
   macro avg      0.400     0.452     0.400       170
weighted avg      0.752     0.618     0.669       170

0.6176470588235294
Wall time: 316 ms


In [30]:
%%time
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred_values_dt = model.predict(X_test)
show_result(pred_values_dt, y_test)
save_res('DecisionTree', feature_type, target_type, pred_values_dt, y_test, len(res))

[[76 55  9]
 [ 8  5  1]
 [11  0  5]]
              precision    recall  f1-score   support

           0      0.800     0.543     0.647       140
           1      0.083     0.357     0.135        14
           2      0.333     0.312     0.323        16

    accuracy                          0.506       170
   macro avg      0.406     0.404     0.368       170
weighted avg      0.697     0.506     0.574       170

0.5058823529411764
Wall time: 834 ms


In [31]:
%%time
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
pred_values_gb = model.predict(X_test)
show_result(pred_values_gb, y_test)
save_res('GradientBoosting', feature_type, target_type, pred_values_gb, y_test, len(res))

[[108  13  19]
 [  8   3   3]
 [  9   0   7]]
              precision    recall  f1-score   support

           0      0.864     0.771     0.815       140
           1      0.188     0.214     0.200        14
           2      0.241     0.438     0.311        16

    accuracy                          0.694       170
   macro avg      0.431     0.474     0.442       170
weighted avg      0.750     0.694     0.717       170

0.6941176470588235
Wall time: 30 s


In [32]:
import os
os.remove("res.xlsx") 

res.to_excel('res.xlsx', index=False)